# Блок 1. Импорт

In [1]:
import ta
import math
import numpy as np
import pandas as pd
import ruptures as rpt
import pandas_profiling
from tqdm import tqdm
from copy import deepcopy
from sklearn.mixture import GaussianMixture

In [2]:
import cufflinks as cf
import plotly.offline
import plotly.express as px
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [4]:
path = '/home/brainiac/Vlad/Git Files/CryptoLp DataScience/DataSets/'
file = 'Bars_700t_Exp_Profits'

In [5]:
df_data = pd.read_csv(path+file+'.csv')
df_data = df_data.dropna()
df_data.index = range(len(df_data))

In [6]:
df_data.tail()

,datetime,open,low,high,close,volume,Naive_Strategy_Profits,Naive_Hedge_Strategy_Profits,Ml_Hedge_Strategy_Profits
485200,2020-05-17 23:47:01.913122,9681.0,9674.5,9686.0,9685.5,2465104,-5.39342,0,0
485201,2020-05-17 23:50:58.915882,9685.5,9667.5,9685.5,9667.5,2246675,18.101,12.4623,0
485202,2020-05-17 23:52:54.954397,9667.5,9658.0,9668.0,9658.5,1366501,8.62899,11.1655,0
485203,2020-05-17 23:55:48.403255,9658.5,9658.0,9668.0,9662.0,2329572,-8.75398,-11.4119,0
485204,2020-05-17 23:59:42.672295,9662.0,9661.5,9674.5,9674.5,1767495,-5.82932,6.59536,0


# Блок 2. Основные функции

In [7]:
# Вычисление возвратов (Не используется)
def calculation_returns(df_data_close_price:pd.Series):
    returns = df_data_close_price.pct_change()*100
    return returns

In [8]:
# Вычисление стандартной волатильности
def calculation_volatility_STD(df_data_close_price:pd.Series,
                               win_size):
    volatility_STD = df_data_close_price.pct_change().rolling(win_size).std()*100
    return volatility_STD

In [9]:
# Кластеризация
def clustering_GaussianHMM(df_data:pd.DataFrame,
                           n_components_:int,
                          ):
    model = GaussianMixture(n_components=n_components_, covariance_type="full")
    model.fit(df_data)
    marked_regimes = model.predict(df_data)
    return marked_regimes

In [10]:
# Построение графиков
def show_plot_plotly(df_data:pd.DataFrame,
                     left_border,
                     right_border,
                     x_,
                     y_,
                     color_:str,
                     title_:str,             
             ):  
    fig = px.scatter(df_data[left_border:right_border], 
                 x=x_, 
                 y=y_, 
                 color=color_, 
                 #color_discrete_sequence=['blue','darkorange','green','red','gold', 'black'], 
                 title=title_
                )    
    return fig

# Блок 3. Возможно нужные функции

In [11]:
df_data.head(1)

,datetime,open,low,high,close,volume,Naive_Strategy_Profits,Naive_Hedge_Strategy_Profits,Ml_Hedge_Strategy_Profits
0,2019-01-01 00:02:29.356431,3693.0,3690.5,3694.5,3690.5,1659932,-,-,-


In [12]:
high = df_data['high']
low = df_data['low']
close = df_data['close']
volume = df_data['volume']

### Momentum Indicators

In [13]:
%%time
momentum_ao = ta.momentum.ao(high, low, s=5, len=34, fillna=False)
momentum_kama = ta.momentum.kama(close, n=10, pow1=2, pow2=30, fillna=False)
momentum_roc = ta.momentum.roc(close, n=12, fillna=False)
momentum_rsi = ta.momentum.rsi(close, n=14, fillna=False)
momentum_stoch = ta.momentum.stoch(high, low, close, n=14, d_n=3, fillna=False)
momentum_tsi = ta.momentum.tsi(close, r=25, s=13, fillna=False)
momentum_uo = ta.momentum.uo(high, low, close, s=7, m=14, len=28, ws=4.0, wm=2.0, wl=1.0, fillna=False)
momentum_wr = ta.momentum.wr(high, low, close, lbp=14, fillna=False)

CPU times: user 23.2 s, sys: 181 ms, total: 23.4 s
Wall time: 23.1 s


### Volume Indicators

In [14]:
%%time
volume_acc_dist_index = ta.volume.acc_dist_index(high, low, close, volume, fillna=False)
volume_chaikin_money_flow = ta.volume.chaikin_money_flow(high, low, close, volume, n=20, fillna=False)
volume_ease_of_movement = ta.volume.ease_of_movement(high, low, volume, n=14, fillna=False)
volume_force_index = ta.volume.force_index(close, volume, n=13, fillna=False)
volume_money_flow_index = ta.volume.money_flow_index(high, low, close, volume, n=14, fillna=False)
volume_negative_volume_index = ta.volume.negative_volume_index(close, volume, fillna=False)
volume_on_balance_volume = ta.volume.on_balance_volume(close, volume, fillna=False)
volume_sma_ease_of_movement = ta.volume.sma_ease_of_movement(high, low, volume, n=14, fillna=False)
volume_volume_price_trend = ta.volume.volume_price_trend(close, volume, fillna=False)
volume_volume_weighted_average_price = ta.volume.volume_weighted_average_price(high, low, close, volume, n=14, fillna= False)

CPU times: user 33.5 s, sys: 15.5 ms, total: 33.6 s
Wall time: 33.5 s


### Trend Indicators

In [15]:
%%time
trend_adx = ta.trend.adx(high, low, close, n=14, fillna=False)
trend_adx_neg = ta.trend.adx_neg(high, low, close, n=14, fillna=False)
trend_adx_pos = ta.trend.adx_pos(high, low, close, n=14, fillna=False)
trend_aroon_down = ta.trend.aroon_down(close, n=25, fillna=False)
trend_cci = ta.trend.cci(high, low, close, n=20, c=0.015, fillna=False)
trend_dpo = ta.trend.dpo(close, n=20, fillna=False)
trend_ema_indicator = ta.trend.ema_indicator(close, n=12, fillna=False)
trend_ichimoku_a = ta.trend.ichimoku_a(high, low, n1=9, n2=26, visual=False, fillna=False)
trend_macd = ta.trend.macd(close, n_slow=26, n_fast=12, fillna=False) 
trend_mass_index = ta.trend.mass_index(high, low, n=9, n2=25, fillna=False)
trend_psar_down = ta.trend.psar_down(high, low, close, step=0.02, max_step=0.2, fillna=False)
trend_sma_indicator = ta.trend.sma_indicator(close, n=12, fillna=False)
trend_trix = ta.trend.trix(close, n=15, fillna=False)
trend_vortex_indicator_neg = ta.trend.vortex_indicator_neg(high, low, close, n=14, fillna=False)

CPU times: user 2min 20s, sys: 91.7 ms, total: 2min 20s
Wall time: 2min 20s


### Others

In [16]:
# %%time
# from scipy.stats import kurtosis,skew, boxcox, norm 
# l_kurtosis = []
# l_skewness = []
# price = df_data['close'].values
# for i in tqdm(range(len(price)+100)):
#     l_kurtosis.append(kurtosis(price[i:i+100]))
#     l_skewness.append(skew(price[i:i+100]))
# df_data['kurtosis'] = l_kurtosis[:(len(l_kurtosis)-100)]
# df_data['skewness'] = l_skewness[:(len(l_skewness)-100)]

In [17]:
# Блок 4. Получение новых признаков

In [18]:
# %%time
# Returns = calculation_returns(df_data['close'])
# Structural_Breaks = detect_structural_breaks(df_data_close_price=df_data['close'], batch_size=20_000)

In [19]:
%%time
param_Vol_Std = [1000]
dict_vol_Std = dict.fromkeys(param_Vol_Std)
for i in dict_vol_Std.keys():
    dict_vol_Std[i] = calculation_volatility_STD(df_data_close_price=df_data['close'],
                                                 win_size=i)  

CPU times: user 20.6 ms, sys: 1 µs, total: 20.6 ms
Wall time: 17.1 ms


In [20]:
df_data_extended = deepcopy(df_data)
df_data_extended['momentum_roc'] = momentum_roc
df_data_extended['momentum_uo'] = momentum_uo

#df_data_extended['momentum_ao'] = momentum_ao
#df_data_extended['momentum_kama'] = momentum_kama
#df_data_extended['momentum_stoch'] = momentum_stoch
#df_data_extended['momentum_tsi'] = momentum_tsi
#df_data_extended['momentum_wr'] = momentum_wr
#df_data_extended['momentum_rsi'] = momentum_rsi

df_data_extended['trend_cci'] = trend_cci
df_data_extended['trend_psar_down'] = trend_psar_down

#df_data_extended['trend_adx'] = trend_adx
#df_data_extended['trend_adx_neg'] = trend_adx_neg
#df_data_extended['trend_adx_pos'] = trend_adx_pos
#df_data_extended['trend_aroon_down'] = trend_aroon_down
#df_data_extended['trend_dpo'] = trend_dpo
#df_data_extended['trend_ema_indicator'] = trend_ema_indicator
#df_data_extended['trend_ichimoku_a'] = trend_ichimoku_a
#df_data_extended['trend_macd'] = trend_macd
#df_data_extended['trend_mass_index'] = trend_mass_index
#df_data_extended['trend_sma_indicator'] = trend_sma_indicator
#df_data_extended['trend_trix'] = trend_trix
#df_data_extended['trend_vortex_indicator_neg'] = trend_vortex_indicator_neg

#df_data_extended['volume_acc_dist_index'] = volume_acc_dist_index
#df_data_extended['volume_chaikin_money_flow'] = volume_chaikin_money_flow
#df_data_extended['volume_ease_of_movement'] = volume_ease_of_movement
#df_data_extended['volume_force_index'] = volume_force_index
#df_data_extended['volume_money_flow_index'] = volume_money_flow_index
#df_data_extended['volume_negative_volume_index'] = volume_negative_volume_index
#df_data_extended['volume_on_balance_volume'] = volume_on_balance_volume
#df_data_extended['volume_sma_ease_of_movement'] = volume_sma_ease_of_movement
#df_data_extended['volume_volume_price_trend'] = volume_volume_price_trend
#df_data_extended['volume_volume_weighted_average_price'] = volume_volume_weighted_average_price

In [21]:
# df_data_extended['Returns'] = Returns
# df_data_extended['SB'] = Structural_Breaks
for i in dict_vol_Std.keys():
    df_data_extended['Volatility_STD'+str(i)] = dict_vol_Std[i]
df_data_extended = df_data_extended.dropna()
df_data_extended.index = range(len(df_data_extended))

In [22]:
df_data_extended.tail(3)

,datetime,open,low,high,close,volume,Naive_Strategy_Profits,Naive_Hedge_Strategy_Profits,Ml_Hedge_Strategy_Profits,momentum_roc,momentum_uo,trend_cci,trend_psar_down,Volatility_STD1000
235880,2020-05-17 23:52:54.954397,9667.5,9658.0,9668.0,9658.5,1366501,8.62899,11.1655,0,-0.622492,43.016940,-153.213239,9701.194382,0.103486
235881,2020-05-17 23:55:48.403255,9658.5,9658.0,9668.0,9662.0,2329572,-8.75398,-11.4119,0,-0.576250,41.083390,-133.550552,9696.874944,0.103484
235882,2020-05-17 23:59:42.672295,9662.0,9661.5,9674.5,9674.5,1767495,-5.82932,6.59536,0,-0.278307,51.213588,-87.386405,9692.987449,0.103532


# Блок 5. Кластеризация

In [23]:
X=df_data_extended.drop(['datetime',
                         'open',
                         'low',
                         'high',
                         'close',
                         'volume',
                         'Naive_Strategy_Profits',
                         'Naive_Hedge_Strategy_Profits',
                         'Ml_Hedge_Strategy_Profits'], 
                        axis=1)

In [24]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler
X = RobustScaler().fit_transform(X)
X = pd.DataFrame(data=X, columns=df_data_extended.drop(['datetime',
                                                        'open',
                                                        'low',
                                                        'high',
                                                        'close',
                                                        'volume',
                                                        'Naive_Strategy_Profits',
                                                        'Naive_Hedge_Strategy_Profits',
                                                        'Ml_Hedge_Strategy_Profits'], 
                                                       axis=1).columns)

In [25]:
X.head(1)

,momentum_roc,momentum_uo,trend_cci,trend_psar_down,Volatility_STD1000
0,1.901827,0.9668,1.521737,-1.685279,0.469278


In [26]:
%%time
list_comp = [4,5,6,7]

marked_regimes = []
for comp in range(len(list_comp)):
    marked_regimes.append(clustering_GaussianHMM(df_data = X,
                                                 n_components_=list_comp[comp]))

CPU times: user 32.2 s, sys: 47.2 s, total: 1min 19s
Wall time: 14 s


# Блок 6. Результаты

In [27]:
df_result = deepcopy(df_data_extended[['datetime','close','Naive_Strategy_Profits','Naive_Hedge_Strategy_Profits','Ml_Hedge_Strategy_Profits']])
for i in range(len(marked_regimes)):
    df_result[str(list_comp[i])] = marked_regimes[i]

In [28]:
for i in range(len(marked_regimes)):
    df_result[str(list_comp[i])] = df_result[str(list_comp[i])].map({0:'0', 1:'1', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9',})

In [29]:
df_result = df_result[(df_result['datetime']>="2019-12-01")][1:]
df_result['Naive_Strategy_Profits'] = df_result['Naive_Strategy_Profits'].astype('float')
df_result['Naive_Hedge_Strategy_Profits'] = df_result['Naive_Hedge_Strategy_Profits'].astype('float')
df_result['Ml_Hedge_Strategy_Profits'] = df_result['Ml_Hedge_Strategy_Profits'].astype('float')

In [30]:
#df_result = df_result[(df_result['datetime']>="2020-04-01") & (df_result['datetime']<"2020-05-01")]
df_result.index = range(len(df_result))

In [31]:
df_result.tail(2)

,datetime,close,Naive_Strategy_Profits,Naive_Hedge_Strategy_Profits,Ml_Hedge_Strategy_Profits,4,5,6,7
86277,2020-05-17 23:55:48.403255,9662.0,-8.753977,-11.411892,0.0,0,3,5,3
86278,2020-05-17 23:59:42.672295,9674.5,-5.829320,6.595362,0.0,0,3,4,1


In [32]:
df_neg = df_result[df_result['Ml_Hedge_Strategy_Profits']<0]
df_zero = df_result[df_result['Ml_Hedge_Strategy_Profits']==0]
df_pos = df_result[df_result['Ml_Hedge_Strategy_Profits']>0]

len(df_neg)/len(df_result)*100
len(df_zero)/len(df_result)*100
len(df_pos)/len(df_result)*100

13.330010778984459

60.40983321549855

26.26015600551699

In [33]:
count_regimes = '4'
df_regime_0 = df_result[df_result[count_regimes]=='0']
df_regime_1 = df_result[df_result[count_regimes]=='1']
df_regime_2 = df_result[df_result[count_regimes]=='2']
df_regime_3 = df_result[df_result[count_regimes]=='3']
df_regime_4 = df_result[df_result[count_regimes]=='4']
df_regime_5 = df_result[df_result[count_regimes]=='5']
df_regime_6 = df_result[df_result[count_regimes]=='6']


0, [len(df_regime_0[df_regime_0['Ml_Hedge_Strategy_Profits']<0])/len(df_result)*100,
    len(df_regime_0[df_regime_0['Ml_Hedge_Strategy_Profits']>0])/len(df_result)*100,
    len(df_regime_0[df_regime_0['Ml_Hedge_Strategy_Profits']==0])/len(df_result)*100
   ]

1, [len(df_regime_1[df_regime_1['Ml_Hedge_Strategy_Profits']<0])/len(df_result)*100,
    len(df_regime_1[df_regime_1['Ml_Hedge_Strategy_Profits']>0])/len(df_result)*100,
    len(df_regime_1[df_regime_1['Ml_Hedge_Strategy_Profits']==0])/len(df_result)*100
   ]

2, [len(df_regime_2[df_regime_2['Ml_Hedge_Strategy_Profits']<0])/len(df_result)*100,
    len(df_regime_2[df_regime_2['Ml_Hedge_Strategy_Profits']>0])/len(df_result)*100,
    len(df_regime_2[df_regime_2['Ml_Hedge_Strategy_Profits']==0])/len(df_result)*100
   ]

3, [len(df_regime_3[df_regime_3['Ml_Hedge_Strategy_Profits']<0])/len(df_result)*100,
    len(df_regime_3[df_regime_3['Ml_Hedge_Strategy_Profits']>0])/len(df_result)*100,
    len(df_regime_3[df_regime_3['Ml_Hedge_Strategy_Profits']==0])/len(df_result)*100
   ]

4, [len(df_regime_4[df_regime_4['Ml_Hedge_Strategy_Profits']<0])/len(df_result)*100,
    len(df_regime_4[df_regime_4['Ml_Hedge_Strategy_Profits']>0])/len(df_result)*100,
    len(df_regime_4[df_regime_4['Ml_Hedge_Strategy_Profits']==0])/len(df_result)*100
   ] 

5, [len(df_regime_5[df_regime_5['Ml_Hedge_Strategy_Profits']<0])/len(df_result)*100,
    len(df_regime_5[df_regime_5['Ml_Hedge_Strategy_Profits']>0])/len(df_result)*100,
    len(df_regime_5[df_regime_5['Ml_Hedge_Strategy_Profits']==0])/len(df_result)*100
   ] 

6, [len(df_regime_6[df_regime_6['Ml_Hedge_Strategy_Profits']<0])/len(df_result)*100,
    len(df_regime_6[df_regime_6['Ml_Hedge_Strategy_Profits']>0])/len(df_result)*100,
    len(df_regime_6[df_regime_6['Ml_Hedge_Strategy_Profits']==0])/len(df_result)*100
   ] 

(0, [5.008171165637061, 13.500388275246584, 33.64665793530291])

(1, [4.717254488345947, 6.316716698153664, 19.844921707483863])

(2, [0.08576826342447177, 0.1819678021302982, 0.1831268327171154])

(3, [3.5188168615769766, 6.261083229986439, 6.7351267399946675])

(4, [0.0, 0.0, 0.0])

(5, [0.0, 0.0, 0.0])

(6, [0.0, 0.0, 0.0])

In [34]:
(0, [12.101728417245988, 12.313990184297122, 48.970698891521884])
(1, [5.583944475017127, 5.490167450949563, 9.54616412663829])
(2, [0.16677710268303364, 0.16790018081557934, 0.22686178277422758])
(3, [1.5481632057142214, 1.6335171437876932, 2.2500870385552725])
(4, [0.0, 0.0, 0.0])
(5, [0.0, 0.0, 0.0])
(6, [0.0, 0.0, 0.0])

(0, [12.101728417245988, 12.313990184297122, 48.970698891521884])

(1, [5.583944475017127, 5.490167450949563, 9.54616412663829])

(2, [0.16677710268303364, 0.16790018081557934, 0.22686178277422758])

(3, [1.5481632057142214, 1.6335171437876932, 2.2500870385552725])

(4, [0.0, 0.0, 0.0])

(5, [0.0, 0.0, 0.0])

(6, [0.0, 0.0, 0.0])